# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
column_indices = dict(zip(
    ['artist_name','user_first_name','gender','item_in_session','user_last_name','song_length','user_level','user_location','session_id','song_name','user_id'], 
    range(11))
)

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(["127.0.0.1"])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity_project
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")

#### Set Keyspace

In [8]:
session.set_keyspace('udacity_project')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




**Note**: For the next queries, we use lowercase names because cassandra will lowercase the column names anyway and that way when retrieving and iterating over the results, we don't have an inconsistent naming

### Query 1
*Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4*

For this query, we:

- we query by `session_id` and `user_item_in_session`, so we need to include them in the PRIMARY KEY
- those two can also uniquely identify a single row

=> `PRIMARY KEY (session_id, item_in_session)`

In [9]:
session.execute("""
    CREATE TABLE session_library (
        session_id int, 
        item_in_session int,
        artist_name text, 
        song_name text,
        song_length float,
        PRIMARY KEY (session_id, item_in_session)
    )
""")              

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library (session_id, item_in_session, artist_name, song_name, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (
            int(line[column_indices['session_id']]), int(line[column_indices['item_in_session']]),
            line[column_indices['artist_name']], line[column_indices['song_name']], 
            float(line[column_indices['song_length']])
        ))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
result = session.execute("""
    SELECT artist_name, song_name, song_length 
    FROM session_library
    WHERE 
        session_id = 338 
        AND item_in_session = 4
""")
display(pd.DataFrame(list(result)))

,artist_name,song_name,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2
*Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182*

For this query we know that:
- we need to query by the `user_id` and `session_id`, so these need to be part of the primary key
- those two alone **cannot** uniquely identify a row as a single user session can have multiple items
- so we add the `item_in_session` column to the primary key to make it unique
- but it has to be added at the **end**, because we don't want to filter by it

=> `Primary KEY (user_id, session_id, item_in_session)`

In [12]:
session.execute("""
    CREATE TABLE IF NOT EXISTS user_library (
        user_id int,
        session_id int, 
        item_in_session int,
        artist_name text, 
        song_name text,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY (user_id, session_id, item_in_session)
    ) WITH CLUSTERING ORDER BY (session_id ASC, item_in_session ASC) -- not really needed because it's the default, but I prefer to be explicit
""")              

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (user_id, session_id, item_in_session, artist_name, song_name, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (
            int(line[column_indices['user_id']]), int(line[column_indices['session_id']]), 
            int(line[column_indices['item_in_session']]), 
            line[column_indices['artist_name']], line[column_indices['song_name']], 
            line[column_indices['user_first_name']], line[column_indices['user_last_name']]
        ))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
result = session.execute("""
    SELECT artist_name, song_name, user_first_name, user_last_name 
    FROM user_library
    WHERE 
        user_id = 10 
        AND session_id = 182
""")
display(pd.DataFrame(list(result)))

,artist_name,song_name,user_first_name,user_last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3
*Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'*

For this query, we know that:
- we need to filter by the song name, so `song_name` has to be a part of the primary key
- we need to include `user_first_name` and `user_last_name` in our columns
- `song_name` alone is not enough to make the row unique
- we can add session related attributes to make the row unique, but actually that would lead to a lot of duplicate results for this query, because the same user may listen to the same song over and over again
- best is to add an identifier related to users to the primary key, because that's what we are querying for
- `user_first_name`, `user_last_name` are not good candidates for inclusion into the primary key, because multiple users can have the same first name and last name, so we would be overwriting those interactions
- `user_id` would be a good choice. It guarantees that we only select unique users, so even if the first name and last name are the same, we know those are actually unique users. If the same user listened to the same song again and again, he would only be recorded in this table once (older row would be replaced), so we wouldn't have to do a `SELECT DISTINCT`
- `user_id` needs to be after `song_name` because we are not actually going to query by `user_id`

=> `PRIMARY KEY(song_name, user_id)`

**Note:** because of this design choice, the number of rows in this table may be less than what we have in the CSV

In [15]:
session.execute("""
    CREATE TABLE IF NOT EXISTS song_library (
        song_name text,
        user_id int,
        user_first_name text,
        user_last_name text,
        PRIMARY KEY (song_name, user_id)
    )
""")

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (song_name, user_id, user_first_name, user_last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (
            line[column_indices['song_name']], int(line[column_indices['user_id']]),
            line[column_indices['user_first_name']], line[column_indices['user_last_name']]
        ))

#### Do a SELECT to verify that the data have been inserted into each table

In [17]:
result = session.execute("""
    SELECT user_first_name, user_last_name 
    FROM song_library
    WHERE 
        song_name = 'All Hands Against His Own'
""")
display(pd.DataFrame(list(result)))

,user_first_name,user_last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [18]:
session.execute("DROP TABLE IF EXISTS session_library")
session.execute("DROP TABLE IF EXISTS user_library")
session.execute("DROP TABLE IF EXISTS song_library")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()